# SEMA-1D

SEMA-1D is a fine-tuned ESM-1v model aimed to predict epitope resiudes based on antigen protein sequence

The MIT License (MIT)
Copyright (c) 2016 AYLIEN
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Requirements 

In [1]:
import os
# set cuda params
# 'TORCH_HOME'directory will be used to save origenal esm-1v weights
os.environ['TORCH_HOME'] = "../torch_hub"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [5]:
os.environ['CUDA_VISIBLE_DEVICES']

'0'

In [6]:
import scipy
import sklearn
import math

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch import nn

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction

import esm
from esm.pretrained import load_model_and_alphabet_hub

from sklearn.metrics import r2_score, mean_squared_error

from tqdm import tqdm

## Set model

In [16]:
class ESM1vForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2, pretrained_no = 1):
        super().__init__()
        self.num_labels = num_labels
        self.model_name = "esm1v_t33_650M_UR90S_" + str(pretrained_no) 
        
#         self.esm1v, self.esm1v_alphabet = load_model_and_alphabet_hub(self.model_name) 
        self.esm1v, self.esm1v_alphabet = torch.hub.load("facebook/esm-1b", self.model_name)
        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids):
                
        outputs = self.esm1v.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)

        return SequenceClassifierOutput(logits=logits)

In [17]:
class PDB_Dataset(Dataset):
    def __init__(self, df, label_type ='regression'):
        self.df = df
        _, esm1v_alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()
        self.esm1v_batch_converter = esm1v_alphabet.get_batch_converter()
        self.label_type = label_type

    def __getitem__(self, idx):
        item = {}
        _, _, esm1b_batch_tokens = self.esm1v_batch_converter([('' , ''.join(self.df.iloc[idx,0])[:1022])])
        item['token_ids'] = esm1b_batch_tokens
        item['labels'] = torch.unsqueeze(torch.LongTensor(self.df.iloc[idx, 1][:1022]),0)

        return item

    def __len__(self):
        return len(self.df)

In [18]:
model = ESM1vForTokenClassification(pretrained_no = 1)

HTTPError: HTTP Error 403: rate limit exceeded

## Prediction

In [ ]:
test_set = pd.read_csv('../data/test_set.csv')
test_set = test_set.groupby('pdb_id_chain').agg({'resi_pos': list,
                                 'resi_aa': list,
                                 'contact_number': list}).reset_index()
test_ds = PDB_Dataset(test_set[['resi_aa', 'contact_number']],
                      label_type ='regression')

### one model

In [ ]:
# path to model weights
path = '../models/sema_1d_ft_cn_atom_r1_8.0_r2_16.0_0.pth'

In [ ]:
model.load_state_dict(torch.load(path))
model.eval()
model.cuda()

with torch.no_grad():
    preds=[]
    for it in tqdm(test_ds):
        preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())

In [ ]:
test_set['pred'] = [p for p in preds]
test_set.head()

### emsemble of 5 models

In [ ]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_ft_cn_atom_r1_8.0_r2_16.0_{str(ens_idx)}.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

with torch.no_grad():
    preds=[]
    for it in tqdm(test_ds):
        preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
res.append(preds)

In [ ]:
res = np.mean(np.array(plists), axis = 0).tolist()
test_set['pred'] = [p for p in res]
test_set.head()